In [ ]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences



#Model Creation 1
#Figure out how to do versioning effectively

pathToTokenizedData = "data/tokenizedData/"
pathToGradeData = "data/gradeData/"

modelOutputPath = "models/"

#Load the data
#tokenized data is in tokenizedData.pkl, has tokenizer obj in tokenizer.json

#Load the tokenizer
with open(pathToTokenizedData + "tokenizer.json", "r") as f:
    tokenizer = Tokenizer.from_json(f.read())

#Load the tokenized data
with open(pathToTokenizedData + "tokenizedData.pkl", "rb") as f:
    tokenizedData = pickle.load(f)

#Load the grade data
#gradeData is a dict with keys as the file names and values as the grades
with open(pathToGradeData + "gradeData.pkl", "rb") as f:
    gradeData = pickle.load(f)





